In [1]:
import pickle
import ast
import re
import numpy as np
import pandas as pd
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdFingerprintGenerator
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from numpy import asarray, mean, std
from xgboost import XGBRegressor
from matplotlib import pyplot
from hyperopt import fmin, tpe, hp, STATUS_OK

# XGBoost-specific data preparation

In [2]:
df_all_identified = pd.read_csv('data_RO2/df_ml_ready.csv')

In [3]:
df_all_identified

,reduced_formulas,etl,htl,bandgap,device_stack,pce,etl_SMILES,htl_SMILES
0,Ag20Bi20CsI60,"['TiO2-c', 'TiO2-mp']",['P3HT'],1.86,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",3.530000,"['O=[Ti]=O', 'O=[Ti]=O']",['CCCCCCC1=CSC=C1']
1,Ag20Bi20CsI60,"['TiO2-c', 'TiO2-mp']",['PTB7-th'],1.86,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",3.530000,"['O=[Ti]=O', 'O=[Ti]=O']",['CCC(=CF)COC1=CC=C(C=C1)C23CCC(CC2)(CC3)C(=O)...
2,Ag2BiI5,"['TiO2-c', 'TiO2-mp']",['PTAA'],2.22,"['SLG', 'ITO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",2.600000,"['O=[Ti]=O', 'O=[Ti]=O']",['CC1=CC(=C(C(=C1)C)N(C2=CC=CC=C2)C3=CC=CC=C3)C']
3,Ag3BiI6,"['TiO2-c', 'TiO2-mp']",['P3HT'],1.80,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",2.320000,"['O=[Ti]=O', 'O=[Ti]=O']",['CCCCCCC1=CSC=C1']
4,Ag3BiI6,"['TiO2-c', 'TiO2-mp']",['PTAA'],0.00,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",4.300000,"['O=[Ti]=O', 'O=[Ti]=O']",['CC1=CC(=C(C(=C1)C)N(C2=CC=CC=C2)C3=CC=CC=C3)C']
...,...,...,...,...,...,...,...,...
5419,CsI3Sn,['TiO2-c'],['PTAA'],0.00,"['SLG', 'FTO', 'TiO2-c', 'Perovskite', 'PTAA',...",3.866667,['O=[Ti]=O'],['CC1=CC(=C(C(=C1)C)N(C2=CC=CC=C2)C3=CC=CC=C3)C']
5420,CsI3Sn,"['TiO2-c', 'TiO2-mp']",['PTAA'],0.00,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",3.790000,"['O=[Ti]=O', 'O=[Ti]=O']",['CC1=CC(=C(C(=C1)C)N(C2=CC=CC=C2)C3=CC=CC=C3)C']
5421,CsI3Sn,"['TiO2-c', 'TiO2-mp']",['PTAA'],1.30,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",3.655000,"['O=[Ti]=O', 'O=[Ti]=O']",['CC1=CC(=C(C(=C1)C)N(C2=CC=CC=C2)C3=CC=CC=C3)C']
5422,CsI3Sn,"['TiO2-c', 'TiO2-mp']",['Spiro-MeOTAD'],0.00,"['SLG', 'FTO', 'TiO2-c', 'TiO2-mp', 'Perovskit...",2.230000,"['O=[Ti]=O', 'O=[Ti]=O']",['COC1=CC=C(C=C1)N(C2=CC=C(C=C2)OC)C3=CC4=C(C=...


## Combine SMILES into one string where there are multiple

In [4]:
etl_combined_SMILES = []
htl_combined_SMILES = []

df_all_identified['etl_SMILES'] = df_all_identified['etl_SMILES'].apply(ast.literal_eval)
df_all_identified['htl_SMILES'] = df_all_identified['htl_SMILES'].apply(ast.literal_eval)

for index, row in df_all_identified.iterrows():
    etl_combination = ".".join(row['etl_SMILES'])
    htl_combination = ".".join(row['htl_SMILES'])
    etl_combined_SMILES.append(etl_combination)
    htl_combined_SMILES.append(htl_combination)
df_all_identified['etl_combined_SMILES'] = etl_combined_SMILES
df_all_identified['htl_combined_SMILES'] = htl_combined_SMILES

df_all_identified['etl_combined_SMILES'] = df_all_identified['etl_combined_SMILES'].str.replace('no_ctl', '')
df_all_identified['htl_combined_SMILES'] = df_all_identified['htl_combined_SMILES'].str.replace('no_ctl', '')


In [5]:
# drop device_stack column which is unnecessary here
df_all_identified = df_all_identified.drop(columns=['device_stack'])

In [6]:
def parse_formula(formula):
    matches = re.findall(r'([A-Z][a-z]?)(\d*)', formula)
    element_counts = {}
    for (element, count) in matches:
        if element in element_counts:
            element_counts[element] += int(count) if count else 1
        else:
            element_counts[element] = int(count) if count else 1
    return element_counts

# Unique elements across all formulas
ELEMENTS = ['H', 'C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I', 'B', 'Li', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ti', 'V', 
            'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 
            'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 
            'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Th', 'U']

# Create columns for each element count
for element in ELEMENTS:
    df_all_identified[element] = df_all_identified['reduced_formulas'].apply(lambda x: parse_formula(x).get(element, 0))

    
fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=1024)    
    
def smiles_to_fingerprint(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return np.zeros(fpgen.GetNumBits(),)
    fp = fpgen.GetFingerprint(molecule)
    # Convert to a bit vector
    bit_vector = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, bit_vector)
    return bit_vector


# Create columns for Morgan Fingerprints
etl_fingerprints = df_all_identified['etl_combined_SMILES'].apply(smiles_to_fingerprint)
htl_fingerprints = df_all_identified['htl_combined_SMILES'].apply(smiles_to_fingerprint)

# Convert fingerprints to DataFrame
etl_fingerprint_df = pd.DataFrame(etl_fingerprints.tolist(), columns=[f'ETL_FP_{i}' for i in range(1024)])
htl_fingerprint_df = pd.DataFrame(htl_fingerprints.tolist(), columns=[f'HTL_FP_{i}' for i in range(1024)])

# Combine all features into a single DataFrame
features_df = pd.DataFrame
features_df = pd.concat([df_all_identified.drop(columns=['reduced_formulas',
                                                         'etl', 
                                                         'htl', 
                                                         'etl_SMILES',
                                                         'htl_SMILES',
                                                         'etl_combined_SMILES',
                                                         'htl_combined_SMILES']), etl_fingerprint_df, htl_fingerprint_df], axis=1)

print(len(features_df))
features_df = features_df.dropna()
print(len(features_df))

[13:49:13] WARNING: not removing hydrogen atom without neighbors
[13:49:29] WARNING: not removing hydrogen atom without neighbors
[13:49:29] WARNING: not removing hydrogen atom without neighbors


5424
5424


# XGBoost Predictions
## Absorber only
### Train Test Split
The split is done after shuffling. To ensure the same shuffling in all models, a pre-defined shuffling index is used.
It was originally generated in the GNN file.

In [7]:
X = features_df.iloc[:, :78].drop(columns='pce')
y = features_df['pce']

# load the splitting indices to make sure all models split the same way
with open("data_RO2/index_for_train_test_split.pkl", "rb") as f:
    splitting_indices = pickle.load(f)
    
X = X.iloc[splitting_indices]
y = y.iloc[splitting_indices]

In [8]:
# Train/test/val split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate sizes based on ratios
total_size = len(features_df)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
train_val_size = train_size + val_size
test_size = total_size - train_size - val_size

# Split the data
X_train = X[:train_val_size]
y_train = y[:train_val_size]
X_test = X[train_val_size:]
y_test = y[train_val_size:]

In [9]:
model_abs_only = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_abs_only, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# training
model_abs_only = XGBRegressor(objective='reg:squarederror')
model_abs_only.fit(X_train, y_train)

MAE: -2.922 (0.108)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [10]:
# Make predictions on the test set
y_pred = model_abs_only.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.762
Mean Squared Error (MSE): 12.077
R^2 Score: 0.483


### Bayesian Hyperoptimization

In [11]:
space = {
    'max_depth': hp.quniform('max_depth', 2, 32, 1),
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 50),
    'gamma': hp.uniform('gamma', 0, 5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

def objective(params):
    xgb_model = XGBRegressor(
        max_depth=int(params['max_depth']),  # Ensure max_depth is an integer
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        n_estimators=int(params['n_estimators']),  # Ensure n_estimators is an integer
        gamma=params['gamma'],
        colsample_bytree=params['colsample_bytree']
    )
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = mean_absolute_error(y_test, y_pred)
    return {'loss': score, 'status': STATUS_OK}

best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

100%|██████████| 100/100 [01:33<00:00,  1.06trial/s, best loss: 2.7544747973694994]
Best set of hyperparameters:  {'colsample_bytree': 0.5228652730351819, 'gamma': 0.41089221510317375, 'learning_rate': 0.04700092548152954, 'max_depth': 8.0, 'n_estimators': 550.0, 'subsample': 0.9914954650099952}


In [12]:
best_params = {'colsample_bytree': 0.5228652730351819, 
               'gamma': 0.41089221510317375, 
               'learning_rate': 0.04700092548152954, 
               'max_depth': 8, 
               'n_estimators': 550, 
               'subsample': 0.9914954650099952}

optim_model = XGBRegressor(**best_params)

# Train the model
optim_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = optim_model.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.754
Mean Squared Error (MSE): 11.998
R^2 Score: 0.486


### Save predictions for model comparison ANOVA

In [13]:
XGBoost_predictions = pd.DataFrame()
XGBoost_predictions['predicted'] = list(y_pred)
XGBoost_predictions['true_values'] = list(y_test)

XGBoost_predictions.to_csv('data_RO2/XGBoost_abs_only_predictions.csv', index=False)

### Save model for future use

In [14]:
# Save model
optim_model.save_model('models/xgboost_models/xgboost_absorber_model.json')
# can be loaded with
# optim_model.load_model('models/xgboost_models/xgboost_labels_model.json')

## Absorber and Label Encoding

In [16]:
X = features_df.iloc[:, :78].drop(columns='pce')
y = features_df['pce']

# Get the unique values
etl_values = df_all_identified['etl'].unique()
htl_values = df_all_identified['htl'].unique()

# Create a dictionary mapping each value to an integer
etl_dict = {value: index for index, value in enumerate(etl_values)}
htl_dict = {value: index for index, value in enumerate(htl_values)}

# Replace each value with its corresponding integer
X['etl_encoded'] = df_all_identified['etl'].map(etl_dict)
X['htl_encoded'] = df_all_identified['htl'].map(htl_dict)

### Train Test Split

In [17]:
# put into the pre-defined shuffled order
X = X.iloc[splitting_indices]
y = y.iloc[splitting_indices]

# Train/test/val split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate sizes based on ratios
total_size = len(features_df)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
train_val_size = train_size + val_size
test_size = total_size - train_size - val_size

# Split the data
X_train = X[:train_val_size]
y_train = y[:train_val_size]
X_test = X[train_val_size:]
y_test = y[train_val_size:]

### Training

In [19]:
model_label_encoding = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_label_encoding, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# training
model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

MAE: -2.612 (0.097)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.551
Mean Squared Error (MSE): 10.495
R^2 Score: 0.550


### Bayesian Hyperparameter Optimization

In [21]:
space = {
    'max_depth': hp.quniform('max_depth', 2, 32, 1),
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 50),
    'gamma': hp.uniform('gamma', 0, 5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

def objective(params):
    xgb_model = XGBRegressor(
        max_depth=int(params['max_depth']),  # Ensure max_depth is an integer
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        n_estimators=int(params['n_estimators']),  # Ensure n_estimators is an integer
        gamma=params['gamma'],
        colsample_bytree=params['colsample_bytree']
    )
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = mean_absolute_error(y_test, y_pred)
    return {'loss': score, 'status': STATUS_OK}

best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

100%|██████████| 100/100 [01:43<00:00,  1.03s/trial, best loss: 2.4928134332823215]
Best set of hyperparameters:  {'colsample_bytree': 0.8358819866311915, 'gamma': 1.5707571535921143, 'learning_rate': 0.06662461294109874, 'max_depth': 7.0, 'n_estimators': 750.0, 'subsample': 0.6954963949941932}


In [22]:
best_params = {'colsample_bytree': 0.8358819866311915, 
               'gamma': 1.5707571535921143, 
               'learning_rate': 0.06662461294109874, 
               'max_depth': 7, 
               'n_estimators': 750, 
               'subsample': 0.6954963949941932}

optim_model = XGBRegressor(**best_params)

# Train the model
optim_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = optim_model.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.493
Mean Squared Error (MSE): 10.090
R^2 Score: 0.568


### Save predictions for comparison ANOVA

In [23]:
XGBoost_predictions = pd.DataFrame()
XGBoost_predictions['predicted'] = list(y_pred)
XGBoost_predictions['true_values'] = list(y_test)

XGBoost_predictions.to_csv('data_RO2/XGBoost_labels_predictions.csv', index=False)

### Save model for future use

In [24]:
# Save model
optim_model.save_model('models/xgboost_models/xgboost_labels_model.json')
# can be loaded with
# optim_model.load_model('models/xgboost_models/xgboost_labels_model.json')

## Absorber and Fingerprints
### Train Test Split

In [25]:
X = features_df.drop(columns=['pce'])
y = features_df['pce']

# put into the pre-defined shuffled order
X = X.iloc[splitting_indices]
y = y.iloc[splitting_indices]

# Train/test/val split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate sizes based on ratios
total_size = len(features_df)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
train_val_size = train_size + val_size
test_size = total_size - train_size - val_size

# Split the data
X_train = X[:train_val_size]
y_train = y[:train_val_size]
X_test = X[train_val_size:]
y_test = y[train_val_size:]

### Training

In [26]:
model = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# training
model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

MAE: -2.566 (0.096)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.513
Mean Squared Error (MSE): 10.162
R^2 Score: 0.565


### Bayesian Optimization for Hyperparameters

In [28]:
space = {
    'max_depth': hp.quniform('max_depth', 2, 32, 1),
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 50),
    'gamma': hp.uniform('gamma', 0, 5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

def objective(params):
    xgb_model = XGBRegressor(
        max_depth=int(params['max_depth']),  # Ensure max_depth is an integer
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        n_estimators=int(params['n_estimators']),  # Ensure n_estimators is an integer
        gamma=params['gamma'],
        colsample_bytree=params['colsample_bytree']
    )
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = mean_absolute_error(y_test, y_pred)
    return {'loss': score, 'status': STATUS_OK}

best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

100%|██████████| 100/100 [11:38<00:00,  6.98s/trial, best loss: 2.4485359950979415]
Best set of hyperparameters:  {'colsample_bytree': 0.6619917972589349, 'gamma': 1.9191744681068839, 'learning_rate': 0.03061390509936575, 'max_depth': 9.0, 'n_estimators': 1000.0, 'subsample': 0.5735458208045199}


### Training with optimal values

In [32]:
best_params = {'colsample_bytree': 0.6619917972589349, 
               'gamma': 1.9191744681068839, 
               'learning_rate': 0.03061390509936575, 
               'max_depth': 9, 
               'n_estimators': 1000, 
               'subsample': 0.5735458208045199}

optim_model = XGBRegressor(**best_params)

# Train the model
optim_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = optim_model.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"R^2 Score: {r2:.3f}")

Mean Absolute Error (MAE): 2.449
Mean Squared Error (MSE): 9.869
R^2 Score: 0.577


### Save for comparison ANOVA

In [33]:
XGBoost_predictions = pd.DataFrame()
XGBoost_predictions['predicted'] = list(y_pred)
XGBoost_predictions['true_values'] = list(y_test)

XGBoost_predictions.to_csv('data_RO2/XGBoost_full_predictions.csv', index=False)

### Save model for future use

In [34]:
# Save model
optim_model.save_model('models/xgboost_models/xgboost_full_model.json')
# can be loaded with
# optim_model.load_model('models/xgboost_models/xgboost_labels_model.json')